<a href="https://colab.research.google.com/github/lincolnschick/ML4MC/blob/main/docs/reports/requirement-10-code/hedges_fully_scripted_on_a_fixed_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="text-align: center">
  <img src="https://github.com/KarolisRam/MineRL2021-Intro-baselines/blob/main/img/colab_banner.png?raw=true">
</div>

# Introduction
This notebook is part one of the Intro track baselines for the [MineRL 2021](https://minerl.io/) competition. It provides an easy way to try out the MineRL environment without having to install anything on your machine. The notebook should run in 5-10 minutes if you click `Runtime -> Run All` in the menu above.  

If you want to edit the notebook, you will have to save a copy to your Google Drive. Simply click `File -> Save a copy in Drive`.

Below you will find a fully scripted agent that spawns in a fixed world and executes a sequence of actions to acquire 6 pieces of wood, craft a wooden pickaxe and dig some cobblestone.  

You can try adjusting the scripted part to progress further in the task and acquire more rewards. Please note that in the competition setting your agent will spawn in a random world - this is explored in the second notebook:  

[MineRL fully scripted](https://colab.research.google.com/drive/1ipj34U_Ub8IsTO0I80o4bUTtltERMErm?usp=sharing)

# Setup

In [ ]:
# %%capture
# # ^ hides output
# !sudo add-apt-repository -y ppa:openjdk-r/ppa
# !sudo apt-get purge openjdk-*
# !sudo apt-get install openjdk-8-jdk
# !sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg

In [ ]:
%%capture
!sudo add-apt-repository -y ppa:openjdk-r/ppa
!sudo apt-get purge openjdk-*
!sudo apt-get install openjdk-8-jdk
!sudo apt-get install xvfb
!sudo apt-get install xserver-xephyr
!sudo apt-get install -y python3-opengl
!sudo apt-get install ffmpeg
!pip3 install gym==0.13.1
!pip3 install minerl==0.4.4
!pip3 install pyvirtualdisplay
!pip3 install -U colabgymrender
!sudo apt-get install xvfb

In [ ]:
!sudo apt-get install xvfb xserver-xephyr vnc4server python-opengl ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package vnc4server is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'vnc4server' has no installation candidate
E: Unable to locate package python-opengl


In [ ]:
%%capture
# ^ hides output
# !pip3 install --upgrade minerl
!pip3 install pyvirtualdisplay
!pip3 install imageio==2.4.1
!pip3 install -U colabgymrender

# Import libraries

In [ ]:
import gym
import minerl
from tqdm.notebook import tqdm
from colabgymrender.recorder import Recorder
from pyvirtualdisplay import Display
import logging
logging.disable(logging.ERROR) # reduce clutter, remove if something doesn't work to see the error logs.

# Start of the agent code

In [ ]:
def str_to_act(env, actions):
    """
    Simplifies specifying actions for the scripted part of the agent.
    Some examples for a string with a single action:
        'craft:planks'
        'camera:[10,0]'
        'attack'
        'jump'
        ''
    There should be no spaces in single actions, as we use spaces to separate actions with multiple "buttons" pressed:
        'attack sprint forward'
        'forward camera:[0,10]'

    :param env: base MineRL environment.
    :param actions: string of actions.
    :return: dict action, compatible with the base MineRL environment.
    """
    act = env.action_space.noop()
    for action in actions.split():
        if ":" in action:
            k, v = action.split(':')
            if k == 'camera':
                act[k] = eval(v)
            else:
                act[k] = v
        else:
            act[action] = 1
    return act

# Actions
Here's a list of all possible actions:
```
Dict(attack:Discrete(2),
     back:Discrete(2),
     camera:Box(low=-180.0, high=180.0, shape=(2,)),
     craft:Enum(crafting_table,none,planks,stick,torch),
     equip:Enum(air,iron_axe,iron_pickaxe,none,stone_axe,stone_pickaxe,wooden_axe,wooden_pickaxe),
     forward:Discrete(2),
     jump:Discrete(2),
     left:Discrete(2),
     nearbyCraft:Enum(furnace,iron_axe,iron_pickaxe,none,stone_axe,stone_pickaxe,wooden_axe,wooden_pickaxe),
     nearbySmelt:Enum(coal,iron_ingot,none),
     place:Enum(cobblestone,crafting_table,dirt,furnace,none,stone,torch),
     right:Discrete(2),
     sneak:Discrete(2),
     sprint:Discrete(2))
```

### Camera
Camera actions contain two values:
1. Pitch (up/down), where up is negative, down is positive.
2. Yaw (left/right), where left is negative, right is positive.  

For example, moving the camera up by 10 degrees would be 'camera:[-10,0]'.


# Change agent behaviour here
To change the sequence of actions that the agent performs, change the code inside the `get_action_sequence()` function below. One action is done every tick and there are 20 ticks per second in a regular Minecraft game.  


In [ ]:
def get_action_sequence():
    """
    Specify the action sequence for the agent to execute.
    """
    # get 6 logs:
    action_sequence = []
    action_sequence += [''] * 100  # wait 5 sec
    action_sequence += ['forward'] * 8
    action_sequence += ['attack'] * 61
    action_sequence += ['camera:[-10,0]'] * 7  # look up
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += [''] * 50
    action_sequence += ['jump']
    action_sequence += ['forward'] * 10
    action_sequence += ['camera:[-10,0]'] * 2
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['attack'] * 61
    action_sequence += ['camera:[10,0]'] * 9  # look down
    action_sequence += [''] * 50

    # make planks, sticks, crafting table and wooden pickaxe:
    action_sequence += ['back'] * 2
    action_sequence += ['craft:planks'] * 4
    action_sequence += ['craft:stick'] * 2
    action_sequence += ['craft:crafting_table']
    action_sequence += ['camera:[10,0]'] * 9
    action_sequence += ['jump']
    action_sequence += [''] * 5
    action_sequence += ['place:crafting_table']
    action_sequence += [''] * 10

    # bug: looking straight down at a crafting table doesn't let you craft. So we look up a bit before crafting:
    action_sequence += ['camera:[-1,0]']
    action_sequence += ['nearbyCraft:wooden_pickaxe']
    action_sequence += ['camera:[1,0]']
    action_sequence += [''] * 10
    action_sequence += ['equip:wooden_pickaxe']
    action_sequence += [''] * 10

    # dig down:
    action_sequence += ['attack'] * 600
    action_sequence += [''] * 10

    # make stone pick and furnace
    action_sequence += ['jump']
    action_sequence += [''] * 5
    action_sequence += ['place:crafting_table']
    action_sequence += [''] * 10

    action_sequence += ['camera:[-1,0]']
    action_sequence += ['nearbyCraft:stone_pickaxe']
    action_sequence += ['nearbyCraft:furnace']
    action_sequence += ['camera:[1,0]']
    action_sequence += [''] * 10
    action_sequence += ['equip:stone_pickaxe']
    action_sequence += [''] * 10

    return action_sequence

# Start Minecraft

In [ ]:
display = Display(visible=0, size=(400, 300))
display.start();

In [ ]:
!sudo apt install tigervnc-standalone-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfile-readbackwards-perl tigervnc-common tigervnc-tools x11-xserver-utils
Suggested packages:
  xfonts-100dpi | xfonts-75dpi xfonts-scalable nickle cairo-5c xorg-docs-core
The following NEW packages will be installed:
  libfile-readbackwards-perl tigervnc-common tigervnc-standalone-server
  tigervnc-tools x11-xserver-utils
0 upgraded, 5 newly installed, 0 to remove and 18 not upgraded.
Need to get 1,442 kB of archives.
After this operation, 3,886 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libfile-readbackwards-perl all 1.06-1 [11.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tigervnc-common amd64 1.12.0+dfsg-4 [101 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tigervnc-standalone-server amd64 1.12.0+dfsg-4 [1,138 kB]
Get:4 http://archive.u

In [ ]:
env = gym.make('MineRLObtainDiamond-v0')
env = Recorder(env, './video', fps=60)

# Run your agent
After the code below finishes you should see a video of the agent and a line saying it received 35.0 reward.

In [ ]:
action_sequence = get_action_sequence()

env.seed(21)
obs = env.reset();

total_reward = 0
for i, action in enumerate(tqdm(action_sequence)):
    obs, reward, done, _ = env.step(str_to_act(env, action))
    total_reward += reward
    if done:
        break

env.release()
# env.play()
env.close()

print(f'\nTotal reward = {total_reward}')

  0%|          | 0/1436 [00:00<?, ?it/s]


Total reward = 99.0


In [ ]:
for o, n in obs['inventory'].items():
    print(o, n)

coal 0
cobblestone 5
crafting_table 0
dirt 3
furnace 1
iron_axe 0
iron_ingot 0
iron_ore 0
iron_pickaxe 0
log 2
planks 5
stick 4
stone 0
stone_axe 0
stone_pickaxe 1
torch 0
wooden_axe 0
wooden_pickaxe 1
